In [1]:
! pip install kaggle

In [10]:
!mkdir -p ~/kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [11]:
# Api to fetch dataset from kaggle
! kaggle datasets download -d kazanova/sentiment140

sentiment140.zip: Skipping, found more recently modified local copy (use --force to force download)


In [12]:
# extracting the compress dataset
from zipfile import ZipFile
dataset='/content/sentiment140.zip'
with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print('the dataset is extracted')

the dataset is extracted


In [13]:
# importing the dependiceis
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


In [14]:
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

# data processing


In [15]:
# loading the data frame from csv to pandas
twitter_data=pd.read_csv('/content/training.1600000.processed.noemoticon.csv',encoding='ISO-8859-1')


In [16]:
twitter_data.shape

(1599999, 6)

In [17]:
twitter_data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [18]:
# naming the columns and reading the dataset again
column_names=['target','id','data','flag','user','text']
twitter_data=pd.read_csv('/content/training.1600000.processed.noemoticon.csv',names=column_names,encoding='ISO-8859-1')


In [21]:
twitter_data.sample(5)

,target,id,data,flag,user,text
1151420,4,1978566021,Sat May 30 23:45:26 PDT 2009,NO_QUERY,degrassifanx,@debbiedeezy finally found your twitter
1591740,4,2191615936,Tue Jun 16 05:38:25 PDT 2009,NO_QUERY,propagation0,"@dave1022 Yeah but oh well...That was then, th..."
306281,0,2000189340,Mon Jun 01 21:35:25 PDT 2009,NO_QUERY,BeantownCutie,that was def bad
725381,0,2262308657,Sat Jun 20 22:15:50 PDT 2009,NO_QUERY,kidfyjy,@theeashlee now I have to find someone to cudd...
129014,0,1835020143,Mon May 18 04:54:54 PDT 2009,NO_QUERY,mamacatscrap,"Ugh, Monday. Miss you @dorkyromeo"


In [22]:
# counting the number of missing values in the dataset
twitter_data.isnull().sum()

target    0
id        0
data      0
flag      0
user      0
text      0
dtype: int64

In [23]:
# checking the sitribution of target column
twitter_data['target'].value_counts()

0    800000
4    800000
Name: target, dtype: int64

convert the target 4 to 1

In [26]:
twitter_data.replace({'target':{4:1}},inplace=True)

In [27]:
twitter_data['target'].value_counts()

0    800000
1    800000
Name: target, dtype: int64

0 ---> negative tweet
1 ---> positive tweet

# Stemming

In [28]:
port_stem=PorterStemmer()

In [32]:
def stemming(content):
  stemmed_content=re.sub('[^a-zA-Z]',' ',content)
  stemmed_content=stemmed_content.lower()
  stemmed_content=stemmed_content.split()
  stemme_content=[port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content=' '.join(stemmed_content)

  return stemmed_content

In [33]:
twitter_data['stemmed_content']=twitter_data['text'].apply(stemming)

In [34]:
twitter_data.head()

,target,id,data,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com y zl awww that s a...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,is upset that he can t update his facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan i dived many times for the ball manag...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass no it s not behaving at all i ...


In [38]:
# separating  data and label
X=twitter_data['stemmed_content'].values
y=twitter_data['target'].values


In [39]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,stratify=y,random_state=2)

In [41]:
# converting the textual data to numerical
vectorizer=TfidfVectorizer()
X_train=vectorizer.fit_transform(X_train)
X_test=vectorizer.transform(X_test)

# training the machine learning model

In [42]:
model=LogisticRegression(max_iter=1000,)

In [45]:
model.fit(X_train,y_train)


LogisticRegression(max_iter=1000)

In [46]:
X_train_prediction=model.predict(X_train)
training_data_accuracy=accuracy_score(y_train,X_train_prediction)
print('Accuracy score on training data:',training_data_accuracy)

Accuracy score on training data: 0.82785


In [48]:
X_test_prediction=model.predict(X_test)
testing_data_accuracy=accuracy_score(y_test,X_test_prediction)
print('Accuracy score on test data:',testing_data_accuracy)

Accuracy score on test data: 0.80215625


In [50]:
import pickle
filename='trained_model.sav'
pickle.dump(model,open(filename,'wb'))


In [51]:
# loading the saved model
loaded_model=pickle.load(open('/content/trained_model.sav','rb'))

In [53]:
X_new=X_test[200]
print(y_test[200])
prediction=loaded_model.predict(X_new)
print(prediction)

if (prediction[0]==0):
  print('Negative tweet')
else:
  print('postive tweet')


1
[1]
postive tweet


In [54]:
X_new=X_test[3]
print(y_test[3])
prediction=loaded_model.predict(X_new)
print(prediction)

if (prediction[0]==0):
  print('Negative tweet')
else:
  print('postive tweet')


0
[0]
Negative tweet
